## **AUTHOR RECOGNITION**

In [ ]:
import numpy as np
import pandas as pd
import string
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

In [ ]:
#DATASET spooky authors

#df=pd.read_csv('/content/top7_EL.csv')
#df=pd.read_csv('/content/train.csv')
df=pd.read_csv('/content/top11_EL.csv')

In [ ]:
df.head()

In [ ]:
df["author"].unique()

In [ ]:
#df=df[df["author"].isin(["Mary Wollstonecraft Shelley","Howard Phillips Lovecraft","Edgar Allan Poe"])]

In [ ]:
df.index=range(len(df))

In [ ]:
df.groupby("author").count()

### **PREPROCESSING:clean and tokenize**

TOKENIZZAZIONE PER PAROLE:un testo può essere concepito come lista di frasi come lista di stringhe(parole,punteggiatura,numeri...).La tokenizzazione per parole prende un testo e lo trasforma in una lista di stringhe dette token.

In [ ]:
nltk.download('punkt')

df['tokenized_sents'] = df.apply(lambda row: nltk.word_tokenize(row["text"]), axis=1)

In [ ]:
df.head()

In [ ]:
type(df.loc[10,"tokenized_sents"])

In [ ]:
len(df)

In [ ]:
type(list(df["tokenized_sents"]))

In [ ]:
print(len(list(df["tokenized_sents"])))

RIMOZIONE DELLE STOPWORDS:un testo potrebbe contenere parole che ai fini di una determinata analisi potrebbero non servire e quindi conviene eliminarle.
 

In [ ]:
nltk.download('stopwords')

noises_list=list(string.punctuation)+["-"+"--"+"\n"+" "+'""'+""]
stop_list = stopwords.words('english')+list(string.punctuation) #noise removal:insieme alle stopwords si elimina la punteggiatura


print(stop_list)
print(noises_list)

In [ ]:
df['tokenized_sents'] = df['tokenized_sents'].apply(lambda x: ' '.join([token for token in x if token not in (noises_list)]))

In [ ]:
df.head()

In [ ]:
type(df.loc[10,"tokenized_sents"])

TRASFORMAZIONE IN LOWERCASE DI TUTTE LE PAROLE

In [ ]:
df['tokenized_sents'] = df['tokenized_sents'].apply(lambda x: x.lower())


In [ ]:
df.head()

In [ ]:
type(df.loc[10,"tokenized_sents"])

# **ANALISI DEL DATASET**

In [ ]:
authors=df["author"].unique()
a=np.zeros(len(authors))
d = {'Authors': authors, 'NumberOfTexts': a}
stats = pd.DataFrame(data=d)
for i in range(len(authors)):
  stats.loc[stats["Authors"]==authors[i],"NumberOfTexts"]+=len(df.loc[df["author"]==authors[i]])

stats.head()

In [ ]:
for i in range(len(stats)):
  stats.loc[i,"NumberOfTexts"]=(stats.loc[i,"NumberOfTexts"]/len(df))*100

Oss:il dataset è abbastanza bilanciato

In [ ]:
print("Autori con rispettiva percentuale delle opere nel dataset:")
stats.head()

# **Vettorizzazione con BOW**
L'algoritmo di vettorizzazione Bag-Of-Words si occupa di trasformare un testo suddiviso in token in un vettore di dati numerici.
L'algoritmo va a traformare un dato in un vettore di caratteristiche(features).
Nell'algoritmo BOW le features sono rappresentate dai token.
Viene creato un vocabolario che assegna ad ogni token che in questo caso è una feature un intero.Questo intero rappresenta l'indice della feature all'interno del vettore rappresentante il dato.
Ogni dato verrà codificato con un vettore i cui valori sono impostati rispetto a questo dizionario.
Quindi il vettore corrispondente ad un dato avrà in corrispondenza di ogni indice il numero di occorrenze di quella parola basandosi sul dizionario che ha creato.
Con l'algoritmo BOW non viene codificato l'ordine dei token all'interno di un dato ma solo le occorrenze.
Quindi c'è una perdita di informazione significativa legata all'ordine dei token all'interno della frase.

In [ ]:
#trasformazione delle features da stringhe a numeri
#memo:le features sono i testi

#Verrà usato l'algoritmo BOW

import numpy as np


from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
#
# Create sample set of documents
#
docs = np.array(df["tokenized_sents"])

#
# Fit the bag-of-words model
#
bag = vectorizer.fit_transform(docs)
#
# Get unique words / tokens found in all the documents. The unique words / tokens represents
# the features
#
print(vectorizer.get_feature_names())
#
# Associate the indices with each unique word
#
print(vectorizer.vocabulary_)
#
# Print the numerical feature vector
#
print(bag.toarray())

In [ ]:
X=bag.toarray()
print(X.shape)

In [ ]:
Y=np.empty((len(df),1))
Y

In [ ]:
#trasformazione delle classi targhet da stringhe a numeri
authors=df["author"].unique() #lista/insieme degli autori

targets=np.array(df["author"])

for i in range (len(df)):
  Y[i]=np.where(authors == targets[i])[0][0]

Y=Y.astype("int")
Y

Splitting dei vettori delle feautures e dei targhet

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
print(X_train.shape,X_test.shape)
print(Y_train.shape,Y_test.shape)

In [ ]:
print("Xtrain",type(X_train))
print("Ytrain",type(Y_train))
print("Xtest",type(X_test))
print("Ytest",type(Y_test))
print("Xtrain",X_train.dtype)
print("Ytrain",Y_train.dtype)
print("Xtest",X_test.dtype)
print("Ytest",Y_test.dtype)


**Trasformazione degli array numpy in tensori pytorch**

In [ ]:
import torch

X_train = torch.from_numpy(X_train).float()
Y_train = torch.from_numpy(Y_train).float()

X_test = torch.from_numpy(X_test).float()
Y_test = torch.from_numpy(Y_test).float()

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

# Creazione della rete neurale

In [ ]:
#il numero di neuroni viene fatto corrispondere al numero di colonne di un vettore rappresentante un dato
#cioè in base al numero di termini che hanno una tfidf maggiore di 0

len(X_train[0])

input_layer_neurons=len(X_train[0])
output_layer_neurons=3

print(input_layer_neurons)

print(output_layer_neurons)

In [ ]:
import torch.nn as nn
import torch.nn.functional as fn

#creazione della classe Net
#Net è una sottoclasse di nn.Module quindi eredita tutti i suoi metodi e attributi
class Net(nn.Module):
  def __init__(self): #costruttore della classe
    super().__init__()  #viene richiamato il costruttore della classe nn.Module

    #definizione dei layers(quindi degli strati della rete che avranno un numero di ingressi e un numero di uscite determinati)
    #nn.linear applica una trasformazione lineare ai dati
    self.fc1=nn.Linear(input_layer_neurons,124)  #Input Layer : ha 28*28 ingressi(quindi 784 neuroni) e 64 uscite possibili(cioé gli ingressi del layer successivo saranno 64)
    self.fc2=nn.Linear(124,64)     #Hidden Layer 1: ha 64 ingressi e 64 uscite
    self.fc3=nn.Linear(64,64)     #Hidden Layer 2: ha 64 ingressi e 64 uscite
    self.fc4=nn.Linear(64,output_layer_neurons)     #Output Layer : ha 64 ingressi e 10 uscite perché le classi possibili sono 10
  
  #definizione della logica secondo cui i dati attraversano i vari layer
  #ad ogni dato quando attraversa ogni neurone viene applicata la funzione relu
  def forward(self,x):
    x=fn.relu(self.fc1(x))
    x=fn.relu(self.fc2(x))
    x=fn.relu(self.fc3(x))
    x=self.fc4(x)

    #return x
    return fn.log_softmax(x,dim=1)  #ritorna la distribuzione probabilistica dell'output della rete

In [ ]:
net=Net() #istanziazione della rete neurale

# Caricamento dei dati(comprende la suddivisione in batch)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 10

train_data = TensorDataset(X_train,Y_train)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(X_test,Y_test)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# Addestramento del modello(FASE DI TRAINING)

In [ ]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
  print("gpu available")
else:
  device = torch.device("cpu")

In [ ]:
import torch.optim as optim

#implementazione della loss-function o funzione di costo
#la loss-function calcola quanto gli output della rete neurale si discostano dai valori reali
#il risultato di una loss-function può essere uno scalare(entropia incrociata...) oppure un vettore(one-hot array)

loss_function=nn.CrossEntropyLoss();

#implementazione dell'ottimizzatore 
#l'ottimizzatore si occupa di modificare i pesi dei collegamenti della rete in modo che questa restituisca un output che si avvicini il più possibile al valore corretto
#net.parameters() restituisce un tensore che rappresenta i parametri della rete ossia tutti i pesi della rete
#il learning rate (lr) è un valore che dice di quanto devono essere aggiustati i pesi a ogni iterazione
#Adam è un ottimizzatore che implementa la discesa stocastica del gradiente (Stochastic Gradient Descent)

optimizer=optim.Adam(net.parameters(),lr=0.001)

#l'epoch è rappresenta l'intera fase in cui al modello sono stati passati tutti i dati del trainset suddivisi in batches
#se si effettuano poche epoche il modello non si addestra bene o meglio non aggiusta i propri pesi nel modo migliore
#se si effettuano troppe epoche il modello rischia l'overfitting (si adegua troppo al training set e perde la capacità di generalizzare)

for epoch in range(3):
  for data in train_data:
    X, y = data #suddividiamo l'esempio in features (X) e classe di appartenenza (y)
    #print(X)
    #print(y)
    net.zero_grad() #setta il gradiente a 0
    output=net(X.view(-1,input_layer_neurons))  #ad ogni passo alla rete viene fornito come input l'ouput dell'iterazione precedente
    #print(output)
    loss=fn.nll_loss(output,y.long())  #calcola la loss-function sull'output della rete al passo corrente rispetto al target reale
    loss.backward() #l'errore commesso viene propagato verso ogni collegamento dallo strato di output agli strati più interni(back-propagation)
    optimizer.step()  #basandosi sui valori riportati dal gradiente effettua l'aggiustamento dei parametri della rete in maniera opportuna  
  print(loss) #stampa la loss function dopo ogni epoca

## Valutazione del modello(FASE DI VALIDAZIONE):

In [ ]:
correct=0 #numero di esempi su cui la rete ha predetto in maniera corretta
total=0 #numero di esempi su cui la rete ha effettuato una predizione

with torch.no_grad(): #nella fase di test non dobbiamo aggiustare i pesi o calcolare la loss function percui non ci interessa il gradiente
  for data in test_data:
    X, y = data
    output=net(X.view(-1,input_layer_neurons))
    #print(output)
    for idx, i in enumerate(output):
      #print(torch.argmax(i),y[idx])
      if(torch.argmax(i)==y[idx]):
        correct+=1
      total+=1

print("ACCURACY: ",round(correct/total,3)*100,"%")


RISULTATI:

3 autori: 77.1%, 75.6%, 77.5%  "Mary Wollstonecraft Shelley","Howard Phillips Lovecraft","Edgar Allan Poe"